In [0]:
import mysql.connector
import pandas as pd

cnx = mysql.connector.connect(user='fkAgdxoNxs', 
                              password='GbjW33NDFc',
                              host='remotemysql.com',
                              database='fkAgdxoNxs')
cursor = cnx.cursor()
cursor.execute("select * from Orders;")
orders = cursor.fetchall()

cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Orders' ORDER BY ORDINAL_POSITION ASC")
columns_names = cursor.fetchall()
cnx.close()


orders = pd.DataFrame(orders)
orders.columns = pd.DataFrame(columns_names)[0]

# Pregunta 1:
¿Cómo se distribuye el número de ordenes en función de los paises ("ShipCountry") ?


In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
frecuencias = orders.ShipCountry.value_counts()
frecuencias.plot(kind="bar", grid=True)

In [0]:
fig = plt.figure(figsize=(15,8))
sns.set(style="whitegrid")
sns.barplot(y=frecuencias.index, x=frecuencias.values)
plt.ylabel('Country', fontsize=12)
plt.xlabel('Frequency', fontsize=12)

# Pregunta 2

 ¿Qué país concentra el mayor número de ordenes?  
 (especifique y calcule el estadístico más apropiado)


In [0]:
#frecuencias.index[0]
frecuencias

# Pregunta 3

¿Se puede decir que las ordenes se distribuyen de forma homogénea o heterogénea entre los paises? 
(utilice un indicador cuantitativo para apoyar su respuesta en términos de la variabilidad)


In [0]:
moda = tmp[0]
fmo = moda/tmp.sum()
TV = 1 - fmo

# Pregunta 4

 Muestre cómo se distribuye el número de ordenes en función de los países ("ShipCountry") por cada año de orden ("OrderDate")


In [0]:
def get_year(a):
  return a.year

In [0]:
year_tmp = orders.OrderDate.apply(get_year)
year_tmp.name = 'Year'
year_tmp[0:5]

In [0]:
orders2 = pd.concat([orders[["OrderID","ShipCountry"]],year_tmp],axis=1)
orders2.head()

In [0]:
tabla_freq = orders2.groupby(['ShipCountry','Year']).count()

In [0]:
tabla_freq.head()

In [0]:
tabla_freq.reset_index(inplace=True)
tabla_freq.head()

In [0]:
sns.set(style="whitegrid")
fig = plt.figure(figsize=(15,10))
snsfig = sns.barplot(y='ShipCountry',x='OrderID',hue='Year', data=tabla_freq)

snsfig.axes.set_title("Distribución de las ventas de los países en cada año",fontsize=25)
snsfig.set_xlabel("Frequency",fontsize=20)
snsfig.set_ylabel("Country",fontsize=20)
snsfig.tick_params(labelsize=15)

# Pregunta 5

Basado en el ejemplo inicial, cargue las tablas  "Customers",  "Shippers" y  "Employees" en diferentes variables

In [0]:
cnx = mysql.connector.connect(user='fkAgdxoNxs', password='GbjW33NDFc', host='remotemysql.com', database='fkAgdxoNxs')
cursor = cnx.cursor()
cursor.execute("select * from Customers")
customers = cursor.fetchall()

cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Customers' ORDER BY ORDINAL_POSITION ASC")
columns_names = cursor.fetchall()
cnx.close()


customers = pd.DataFrame(customers)
customers.columns = pd.DataFrame(columns_names)[0]

In [0]:
cnx = mysql.connector.connect(user='fkAgdxoNxs', password='GbjW33NDFc', host='remotemysql.com', database='fkAgdxoNxs')
cursor = cnx.cursor()
cursor.execute("select * from Shippers")
shippers = cursor.fetchall()

cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Shippers' ORDER BY ORDINAL_POSITION ASC")
columns_names = cursor.fetchall()
cnx.close()

shippers = pd.DataFrame(shippers)
shippers.columns = pd.DataFrame(columns_names)[0]

In [0]:
cnx = mysql.connector.connect(user='fkAgdxoNxs', password='GbjW33NDFc', host='remotemysql.com', database='fkAgdxoNxs')
cursor = cnx.cursor()
cursor.execute("select * from Employees")
employees = cursor.fetchall()

cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Employees' ORDER BY ORDINAL_POSITION ASC")
columns_names = cursor.fetchall()
cnx.close()

employees = pd.DataFrame(employees)
employees.columns = pd.DataFrame(columns_names)[0]

# Pregunta 6

Integre las tablas "Orders", "Customers", "Shippers" y "Employees" en un solo data.table

In [0]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
fulldata = pd.merge(orders,customers,on='CustomerID')
fulldata = pd.merge(fulldata,employees,on='EmployeeID')
fulldata = pd.merge(fulldata,shippers, left_on=['ShipVia'], right_on=['ShipperID'])

In [0]:
fulldata.shape

In [0]:
fulldata.head()

In [0]:
fulldata.describe()

In [0]:
fulldata.dtypes

# Pregunta 7

Determine el porcentaje valores faltantes para cada columna
¿Qué estrategia recomendaría particularmente para este caso?


In [0]:
fulldata.count()

In [0]:
fulldata.isna()

In [0]:
freqNA = fulldata.isna().sum()

In [0]:
freqNA = freqNA[freqNA>0]
rowsTotal = fulldata.shape[0]

In [0]:
porcNA = round( (freqNA/rowsTotal) * 100, 2)

In [0]:
# Selecciones las variables (columnas) que tienen más de 10% de valores faltantes
featuresDelete = porcNA.index[porcNA>10]

In [0]:
fulldata2 = fulldata.drop(featuresDelete, axis=1)

freqtmp = fulldata2.isna().sum()
freqtmp[freqtmp>0]

In [0]:
# Vamos a revisar que tipo de datos corresponde las caracteristicas que aún presentan data faltante

fulldata2[freqtmp[freqtmp>0].index].dtypes

En este caso, las caracteristicas son de naturaleza datetime y texto, por lo cual, los metodos de imputación tradicionales no son aplicables. 

Opciones: 

*   Usar un metodo de imputación especial para trabajar con fechas y textos
*   Eliminar esas observaciones (no es la mejor opción, pero podría ser una eliminación temporal)



In [0]:
fulldata2_ = fulldata2.dropna()
fulldata2_.shape

# Pregunta 8
Cree una variable con el monto total $$ de la transacción y otra variable con el número de productos diferentes asociados (tabla "Order Details"), e integrelo al dataframe principal

In [0]:
# Para esta pregunta vamos a trabajar con fulldata

cnx = mysql.connector.connect(user='fkAgdxoNxs', password='GbjW33NDFc', host='remotemysql.com', database='fkAgdxoNxs')
cursor = cnx.cursor()
cursor.execute("SELECT OrderID, SUM(Quantity*UnitPrice) TOTAL FROM `Order Details` GROUP BY OrderID")
total = cursor.fetchall()

cursor.execute("SELECT OrderID, count(*) CANT FROM `Order Details` GROUP BY OrderID")
cantidad = cursor.fetchall()
cnx.close()

total = pd.DataFrame(total)
total.columns = ["OrderID","TOTAL"]

cantidad = pd.DataFrame(cantidad)
cantidad.columns = ["OrderID","CANT"]

In [0]:
fulldata = pd.merge(fulldata,total,on='OrderID')
fulldata = pd.merge(fulldata,cantidad,on='OrderID')

In [0]:
print(fulldata.TOTAL.dtypes)
print(fulldata.CANT.dtypes)

In [0]:
fulldata.TOTAL = fulldata.TOTAL.astype('float')
fulldata.TOTAL.dtypes

# Pregunta 9
¿Hay alguna relación lineal en el monto vendido o número de productos vendidos con respecto al salario del empleado que participo en la transacción?


In [0]:
fig = plt.figure(figsize=(15,5))
sns.scatterplot(x="Salary", y="CANT", data=fulldata )

In [0]:
fig = plt.figure(figsize=(15,5))
sns.scatterplot(x="Salary", y="TOTAL", data=fulldata )

In [0]:
# https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.Timestamp.html

def getYear(fecha):
  return fecha.year

fulldata["Year"] = fulldata.OrderDate.apply(getYear)

In [0]:
fig = plt.figure(figsize=(15,10))
snsfig = sns.scatterplot(x="Salary", y="TOTAL", data=fulldata, hue="ShipCountry", size="CANT", style="Year")

snsfig.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

In [0]:
fulldata[["TOTAL","CANT","Salary"]].corr()

# Pregunta 10

Presente un gráfico que permita visualizar cuánto (frecuencia) vendió cada empleado


In [0]:
freqVentas=fulldata.EmployeeID.value_counts()

In [0]:
freqVentas

In [0]:
indices = list(freqVentas.index)
indices.sort()
indices
freqVentas2 = freqVentas[indices]

In [0]:
plt.figure(figsize=(8,8))
plt.pie(freqVentas2,  labels=freqVentas2.index, autopct='%1.1f%%',counterclock=True)
plt.title("Frecuencia vendida por Empleado",fontsize=20)
_ = plt.legend(fontsize=15, loc='right',bbox_to_anchor=(1.2, 0.5), title="EmployeeID")